In [1]:
import numpy as np
import matplotlib as mpl
import xarray as xr
from collections import OrderedDict

from quantify_core.data.handling import (
    load_dataset,
    set_datadir,
    default_datadir,
)

import time

In [2]:
set_datadir(default_datadir())

Data will be saved in:
/home/pschaefers/quantify-data


In [3]:
test_response = np.array([np.array([0,1,0,1,0,1]),np.array([2,3,2,3,2,3]), np.array([4,5,4,5,4,5]),np.array([6,7,6,7,6,7])])
test_response = test_response.repeat(7,axis=1)
test_response = tuple(test_response)
test_response

(array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]),
 array([2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 3,
        3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3]),
 array([4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 5,
        5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5]),
 array([6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 7,
        7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7]))

In [3]:
# Input params
popsize = 2
repetitions = 3
soft_reps = 1
acquisitions_per_cycle = 7
qubits = ['D1', 'D2']
num_qubits = len(qubits)



In [4]:
12345%512

57

In [3]:
a = np.ones(3)
b = np.ones(3)

print(a+b)

[2. 2. 2.]


In [12]:
maximum = 7*512*30

reps = 513
popsize = 61
total_mem = 7*reps*popsize

print(total_mem) 
print(np.ceil(total_mem/maximum))
print((total_mem % maximum)/(7*reps))

219051
3.0
1.1169590643274854


In [15]:
reps = 1000
popsize = 30

def get_batch_sizes(reps, popsize, max_mem=7*512*30):
    mem_per_individual = 7 * reps
    max_pop_per_batch = int(np.floor(max_mem / mem_per_individual))
    
    batch_sizes = []
    remaining = popsize
    while remaining > 0:
        batch = min(max_pop_per_batch, remaining)
        batch_sizes.append(batch)
        remaining -= batch
    
    return batch_sizes

get_batch_sizes(reps,popsize)

[15, 15]

In [16]:
7*15*1000

105000

In [17]:
7*512*30

107520

In [23]:
population_test = np.array([np.array([1,2,3]),np.array([2,2,3]),np.array([3,2,3]),np.array([4,2,3]), np.array([4,2,3])])

batched = [2,2,1]


def batch_population(population, batched):
    population_batched = []
    start = 0
    for size in batched:
        end = start + size
        population_batched.append(population[start:end])
        start = end
    return population_batched

population_batched = batch_population(population_test, batched)
population_batched[0]

array([1, 2, 3])

In [4]:
tuid = "20250310-133724-427-c941d4" #"20250122-151004-773-a762e7"

dataset = load_dataset(tuid=tuid)
dataset

<xarray.Dataset> Size: 40B
Dimensions:  (dim_0: 2)
Coordinates:
    x0       (dim_0) <U3 24B '010' '000'
Dimensions without coordinates: dim_0
Data variables:
    y0       (dim_0) float64 16B 10.0 0.0
Attributes:
    tuid:                             20250310-133724-427-c941d4
    name:                             test output state from input state
    grid_2d:                          False
    grid_2d_uniformly_spaced:         False
    1d_2_settables_uniformly_spaced:  False

In [30]:
def get_batch_sizes(repetitions, popsize, max_mem=7 * 512 * 30):
    mem_per_individual = 7 * repetitions
    max_pop_per_batch = int(np.floor(max_mem / mem_per_individual))

    batch_sizes = []
    remaining = popsize
    while remaining > 0:
        batch = min(max_pop_per_batch, remaining)
        batch_sizes.append(batch)
        remaining -= batch

    return batch_sizes

def batched_population(population, batch_sizes):
    population_batched = []
    start = 0
    for size in batch_sizes:
        end = start + size
        population_batched.append(population[start:end])
        start = end
    return population_batched

batch_sizes = get_batch_sizes(513, 101)

In [31]:
population = np.arange(513)

pop_batched = batched_population(population, batch_sizes)

for pop in pop_batched:
    print(pop)
    print(len(pop))
    

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28]
29
[29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52
 53 54 55 56 57]
29
[58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81
 82 83 84 85 86]
29
[ 87  88  89  90  91  92  93  94  95  96  97  98  99 100]
14


In [32]:
7*29*513

104139

In [34]:
7*29*512

103936

In [27]:
513*29

14877

In [35]:
101*101

10201

In [2]:
[0.0,0.0,0.0]*9

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [2]:
"""
Note: Before running we need to extract which pulse parameters to run for

TO DO: how long does 1 of these measurements take for all 17 qubits? --> Is 15 mins of waiting time before experiments realistic?
"""
interval_time = 10  # 15 minutes in seconds
total_runtime = 60  # 48 hours in seconds

start_time = time.time()
end_time = start_time + total_runtime
next_run_time = start_time

i=1 

try:
    while time.time() <= end_time:
        print(i)
        i+=1
        # Schedule the next exact run time
        next_run_time += interval_time

        # Sleep until the scheduled time (if needed)
        time_to_wait = next_run_time - time.time()
        if time_to_wait > 0:
            time.sleep(time_to_wait)

except KeyboardInterrupt:
    print("\nStopped by user.")

1
2
3
4
5
6


In [9]:
qubit_names = ["Q1","Q2","Q3"]

def batch_qubits(qubit_names):
    batches = []
    for i in range(0, len(qubit_names), 2):
        batches.append(qubit_names[i:i+2])
    return batches

def batch_data(data, qubits_per_batch=2, values_per_qubit=3):
    batches = []
    # total number of qubits
    num_qubits = len(data) // values_per_qubit

    for i in range(0, num_qubits, qubits_per_batch):
        start = i * values_per_qubit
        end = (i + qubits_per_batch) * values_per_qubit
        batches.append(data[start:end])

    return batches

arr = ["F1", "A1", "D1", "F2", "A2", "D2", "F3", "A3", "D3"]

print(batch_qubit_data(arr))


batched = batch_qubits(qubit_names)

print(batched)
print(len(batched))

[['F1', 'A1', 'D1', 'F2', 'A2', 'D2'], ['F3', 'A3', 'D3']]
[['Q1', 'Q2'], ['Q3']]
2


In [24]:
point = np.array([1,2,3,1,2,3,1,2,3])

print(int(len(point)/3))

3


In [20]:
data =[np.array(["F1", "A1", "D1", "F2", "A2", "D2", "F3", "A3", "D3"]), np.array(["F1", "A1", "D1", "F2", "A2", "D2", "F3", "A3", "D3"])]



[array(['F1', 'A1', 'D1', 'F2', 'A2', 'D2'], dtype='<U2'), array(['F3', 'A3', 'D3'], dtype='<U2')]


In [28]:
a = np.array([])
b = np.array([3,4])

c = np.append(a,b)
print(c)

[3. 4.]
